In [90]:
import Amazon_Movie_Parser as prs
import Graph_Amazon_Movies_2 as gam
#import Computations_debug_2 as comp
import Computations_debug_3 as comp
#import Predict_Movie_v2 as pm
import Predict_Movie_v3 as pm

import numpy as np
import pandas as pd
import os

import networkx as nx
from networkx.algorithms import bipartite
import datetime
#from numpy import savetxt
#import random

pd.set_option('display.max_colwidth', 0)

In [ ]:
# if you run this in colab *** it has a very limited RAM 12GB if I'm not mistaken, so works for 20K not 40K
#from google.colab import drive
#drive.mount('/content/drive')

In [91]:
predict_amazon = pm.Amazon()
#predict_amazon.debug_mode = 'On'
#predict_amazon.file_type = 'pickle'
# **** we need to filter out the movies the person has already reviewed previously ****
# because the same movie can appear in the test data-set since some new users might review it this time

In [116]:
file_name = 'data/movies.txt'; n_movies = 5625; n_movies_v = 20500; n_test_users = 65; walk_steps = 40
start_index_v = n_movies; beta = 0.15; top_neighbor = 20; top_N_movie_suggestions = 1
num_limit=500000; date_year=2009

#file_name = 'data/movies.txt'; n_movies = 2000; n_movies_v = 2000; n_test_users = 40; walk_steps = 30
#start_index_v = n_movies; beta = 0.15; top_neighbor = 50

#lst_n_movies_test = [20000, 30000, 40000, 50000]
#lst_n_movies_test_v = [10000, 13000, 15000, 18000, 20000]
#lst_n_movies_test_v = [3000, 5000, 9000, 14000]
#lst_n_movies_test_v = [2000, 4000, 6000, 8000]
lst_n_movies_test_v = [1400]  # 4038-February

start_time_ALL = datetime.datetime.now()
#for n_movies in lst_n_movies_test:
for n_movies_v in lst_n_movies_test_v:
    start_index_v = n_movies
    start_time_part1 = datetime.datetime.now()

    print('\nn_movies = {} .. n_movies_v = {} .. n_test_users = {}'.format(n_movies, n_movies_v, n_test_users))
    print('walk_steps = {} .. top_neighbor = {}'.format(walk_steps, top_neighbor))
    print('----------------------------------------------')
    print('Calculation starts for above values: {}'.format(start_time_part1))

    max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v, num_limit, date_year)

    end_time_part1 = datetime.datetime.now()
    total_time_part1 = end_time_part1 - start_time_part1
    print('**********************************************')
    print('Calculation time for ALL-Part-1: {}'.format(total_time_part1))
    
    start_time_part2 = datetime.datetime.now()

    '''
    df_FINAL = \
    predict_amazon.run_validation(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor)
    '''
    '''
    df_FINAL = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)
    
    '''
    
    df_FINAL, df_FINAL_all = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)

    end_time_part2 = datetime.datetime.now()
    total_time_part2 = end_time_part2 - start_time_part2
    
    #start_time_part3 = datetime.datetime.now()
    # df_perf is saved in a csv, too, but it i done in the function "save_performance_values"
    df_perf = predict_amazon.save_performance_values(df_FINAL, total_time_part1, total_time_part2)
    
    df_perf_all = predict_amazon.save_performance_values_ALL(df_FINAL_all, total_time_part1, total_time_part2)
    #end_time_part3 = datetime.datetime.now()
    #total_time_part3 = end_time_part3 - start_time_part3

    # just to avoid failure if there is no RESULTS directory
    try:
        file_prefix = "RESULTS/PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "RESULTS/PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
    except:
        file_prefix = "PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
                                                
    #fileName_Details = prs.FileNameUnique(prefix = "RESULTS/PredictionDetails_", suffix = '.csv')
    fileName_Details = prs.FileNameUnique(prefix = file_prefix, suffix = '.csv')
    fileName_Details_all = prs.FileNameUnique(prefix = file_prefix_all, suffix = '.csv')
    
    df_FINAL.to_csv(fileName_Details)
    df_FINAL_all.to_csv(fileName_Details_all)
    print('Prediction Details are saved in {}'.format(fileName_Details))
    print('Prediction_all Details are saved in {}'.format(fileName_Details_all))
    df_perf

end_time_ALL = datetime.datetime.now()
print('Calculation time for ALL-Predictions: {}'.format(end_time_ALL))
os.system('say "your program has finished"')

# it is taking SOOOOOO LONG time, had to stop it (even for 12K edges). Let's try in smaller networks later..
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies)

#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

'''
>>>> df_FINAL (PredictionDetails.csv file)
> prediction: our prediction whether or not the given user will review the given movie on the list or not 
depending on the given threshold
> reality: 
> threshold: if ngbr_ratio > threshold then prediction=1 else 0
> ngbr_ratio: how many percent of your top-K similar users reviewed this movie
> tot_rev_m: how many times was this movie reviewed 
> n_old_edge: how many times did this user make a review in the old dataset
> n_new_edge: how many times did this user make a review in the new dataset
'''


n_movies = 5625 .. n_movies_v = 1400 .. n_test_users = 65
walk_steps = 40 .. top_neighbor = 20
----------------------------------------------
Calculation starts for above values: 2020-06-01 12:24:20.406202
*************** 1st Graph calculations - MODEL *************** 
num_limit=500000, num_reviews=7025, date_year=2009
max_date: 2009-12-31 min_date: 2009-01-01
2009: 7025 .. 2010: 0 .. 2011: 0 .. 2012: 0
Calculation time-Crete graph from the movies file: 0:00:01.987069
Is connected: True ... Is bipartite: True
# of nodes: 3309 ... # of edges: 3918
Calculation time-1st Part: 0:00:02.103864

*************** 2nd Graph calculations - TEST *************** 
Is connected: True ... Is bipartite: True
# of nodes: 657 ... # of edges: 832
n_movies for modeling (P matrix) = 357 .. n_movies for testing = 69
n_users for modeling (P matrix) = 2952 .. n_users for testing = 588
Avg degree (edge) per user in MODELING GRAPH: 1.3272357723577235  ... in TEST GRAPH: 1.4149659863945578
Calculation time-2nd P

'\n>>>> df_FINAL (PredictionDetails.csv file)\n> prediction: our prediction whether or not the given user will review the given movie on the list or not \ndepending on the given threshold\n> reality: \n> threshold: if ngbr_ratio > threshold then prediction=1 else 0\n> ngbr_ratio: how many percent of your top-K similar users reviewed this movie\n> tot_rev_m: how many times was this movie reviewed \n> n_old_edge: how many times did this user make a review in the old dataset\n> n_new_edge: how many times did this user make a review in the new dataset\n'

In [96]:
pd.options.display.max_rows = 1000
predict_amazon.df_summary_ratio_v.sort_values(by='ratio_similar', ascending=False).head(1)['movie'].tolist()

['B0083SI986']

In [ ]:
mv_new='B001NZWEI2'
predict_amazon.df_summary_ratio_v.query("movie=='" + mv_new + "'").values[0].tolist()[1]

In [119]:
df_perf

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.52877,0.000000,0.000000,0,71,Graph Creation,debug_mode,Off,threshold,60
1,,0.000000,0.000000,0.0178571,70,0:00:02.158263,n_movies_model,5625,acc,0.846154
2,,0.111111,0.000000,0.0357143,65,Random Walk & Prediction,n_movies_validation,1400,tpr,0.111111
3,,0.111111,0.333333,0.0714286,60,0:03:20.514540,walk_steps,40,fpr,0.0357143
4,,0.111111,0.166667,0.107143,50,Threshold search,beta,0.15,tnr,0.964286
5,,0.444444,0.142857,0.196429,45,0:00:00.363706,top_neighbor,20,fnr,0.888889
6,,0.444444,0.266667,0.285714,35,TOTAL TIME,n_test_users,65,precision,0.333333
7,,0.444444,0.200000,0.428571,30,0:03:23.036509,threshold,60,f1,0.166667
8,,0.444444,0.142857,0.5,25,,start_index_v,5625,,
9,,0.555556,0.125000,0.607143,20,,file_name,data/movies.txt,,


In [118]:
df_perf_all

,AUC,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.694246,Graph Creation,debug_mode,Off,threshold,0
1,,0:00:02.158263,n_movies_model,5625,acc,
2,,Random Walk & Prediction,n_movies_validation,1400,tpr,
3,,0:03:20.514540,walk_steps,40,fpr,
4,,Threshold search,beta,0.15,tnr,
5,,0:00:00.005410,top_neighbor,20,fnr,
6,,TOTAL TIME,n_test_users,65,precision,
7,,0:03:22.678213,threshold,0,f1,
8,,,start_index_v,5625,,
9,,,file_name,data/movies.txt,,


In [ ]:
df_FINAL

In [ ]:
pd.options.display.max_rows = 1000
df_FINAL

In [7]:
max_connected_gr_amazon_movies_VAL.edges('B00004RNYG')

EdgeDataView([('B00004RNYG', 'A2F3SXHT6RBV81')])

In [ ]:
file_name = 'data/movies.txt'; n_movies = 1000; n_movies_v = 20500; n_test_users = 100; walk_steps = 40
start_index_v = n_movies; beta = 0.1; top_neighbor = 50; top_N_movie_suggestions = 1

#file_name = 'data/movies.txt'; n_movies = 2000; n_movies_v = 2000; n_test_users = 40; walk_steps = 30
#start_index_v = n_movies; beta = 0.15; top_neighbor = 50

#lst_n_movies_test = [20000, 30000, 40000, 50000]
#lst_n_movies_test_v = [10000, 11000, 12000, 13000, 14000]
#lst_n_movies_test_v = [2000, 4000, 6000, 8000]
lst_n_movies_test_v = [1000]

start_time_ALL = datetime.datetime.now()
#for n_movies in lst_n_movies_test:
for n_movies_v in lst_n_movies_test_v:
    start_index_v = n_movies
    start_time_part1 = datetime.datetime.now()

    print('\nn_movies = {} .. n_movies_v = {} .. n_test_users = {}'.format(n_movies, n_movies_v, n_test_users))
    print('walk_steps = {} .. top_neighbor = {}'.format(walk_steps, top_neighbor))
    print('----------------------------------------------')
    print('Calculation starts for above values: {}'.format(start_time_part1))

    max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v)

    end_time_part1 = datetime.datetime.now()
    total_time_part1 = end_time_part1 - start_time_part1
    print('**********************************************')
    print('Calculation time for ALL-Part-1: {}'.format(total_time_part1))

In [ ]:
grpComp = comp.GraphComp()

bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)
bottom_nodes_v, top_nodes_v = bipartite.sets(max_connected_gr_amazon_movies_VAL)

grpComp.movie_user_compare_datasets(top_nodes_v, top_nodes, bottom_nodes_v, bottom_nodes)

In [ ]:
n_test_users=8
df_FINAL, df_FINAL_all = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)

In [ ]:
mv_new = 'B003AI2VGA'
predict_amazon.df_movie_similarity.query("movie_name=='" + mv_new + "'").values[0].tolist()[2]

In [ ]:
row_num = len(predict_amazon.df_movie_similarity)
list(predict_amazon.df_movie_similarity.loc[row_num-1])[2]

In [ ]:
max_connected_gr_amazon_movies.edges('AQP1VPK16SVWM')

In [ ]:
top_N_suggestions=2
top_K_df_movie_similarity = predict_amazon.df_movie_similarity.tail(top_N_suggestions)
lst_top_movie_suggestions = []
for mv in top_K_df_movie_similarity.values:
    lst_top_movie_suggestions.append(mv[1])
    #print(mv)
    
print(lst_top_movie_suggestions)

In [ ]:
predict_amazon.df_movie_similarity

In [ ]:
predict_amazon.df_summary_v

In [ ]:
predict_amazon.df_summary_v

In [ ]:
predict_amazon.R_sorted[0][0]

In [ ]:
list_R = predict_amazon.R_vector.tolist()[0]
for idx in predict_amazon.R_sorted[0][0]:
    print('idx={} .. node={} .. score={}'.format(idx, predict_amazon.node_list[idx], list_R[idx]))

In [ ]:
df_FINAL

In [ ]:
df_FINAL_all

In [ ]:
predict_amazon.node_list[3]

In [ ]:
list(max_connected_gr_amazon_movies.edges(list(bottom_nodes)[0]))

In [ ]:
list(bottom_nodes)[0]

In [ ]:
params_df = predict_amazon.show_parameters_in_use()
params_df.style.set_properties(**{'text-align': 'left'})

df_perf

In [ ]:
df_perf_all

In [ ]:
# it is taking SOOOOOO LONG time
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies_VAL)
#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

# centrality is fast
#centrality = nx.eigenvector_centrality_numpy(max_connected_gr_amazon_movies_VAL)
#print(centrality)

# if we use a small k, it doesn't take much time
#bc = nx.betweenness_centrality(max_connected_gr_amazon_movies_VAL, k=100)
#print(bc)


In [ ]:
len(max_connected_gr_amazon_movies.edges('A1N0LCPR7O7OLL'))

In [ ]:
import statistics as st
import networkx as nx
from networkx.algorithms import bipartite

bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies_VAL)
# top_nodes=movies   ...  bottom_nodes=users
degree_centrality = bipartite.degree_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#betweenness_centrality = bipartite.betweenness_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#closeness_centrality = bipartite.closeness_centrality(max_connected_gr_amazon_movies_VAL, top_nodes)
#st.mean(list(degree_centrality.values()))
st.mean(list(betweenness_centrality.values()))
#---- degree_centrality (normalized) = deg(v)/max_possible_edges
# bottom-nodes: avg normalized-degree_centrality: 0.003492891468893659=VAL  ....  0.001348452313463792
# top-nodes: avg normalized-degree_centrality:    0.003492891468893659=VAL  ....  0.001348452313463792
#---- closeness_centrality
#---- https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.centrality.closeness_centrality.html#networkx.algorithms.bipartite.centrality.closeness_centrality
# bottom-nodes: avg. closeness_centrality: ?=VAL                   ....  ?
# top-nodes: avg. closeness_centrality:    0.3937692751347393=VAL  ....  ?
#---- betweenness_centrality
# even if select bottom or top nodes, it always gives the result for bottom_nodes=USERS
# bottom-nodes: avg. betweenness_centrality: 1.2469622543950363=VAL  (just gave the avg. degree..)  ....  ?
# top-nodes: avg. betweenness_centrality:    1.2469622543950363=VAL  ....  ?
#----
# degree distribution
#degrees(B, nodes[, weight])
# careful here.. if you pass top_nodes as an argument, 
# it returns the values for bottom_nodes as the first result and the top_nodes as the second result.. and vice versa
#degree_distribution_Bottom, degree_distribution_TOP = bipartite.degrees(max_connected_gr_amazon_movies_VAL, top_nodes)
#print(len(degree_distribution_TOP))
#----
#bipartite.color(max_connected_gr_amazon_movies_VAL)
#----
# returns the same for both bottom and top... same value with avg normalized-degree_centrality
#dens_m = bipartite.density(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(dens_m)
#---- clustering coeff for the whole graph
#avg_cluster = bipartite.average_clustering(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(avg_cluster)
# 1.2469622543950363 >> (just gave the avg. degree..)
#----
#n_redundancy = bipartite.node_redundancy(max_connected_gr_amazon_movies_VAL, bottom_nodes)
#print(n_redundancy)
# NetworkXError: Cannot compute redundancy coefficient for a node that has fewer than two neighbors.
print('{} .. {}'.format(len(top_nodes), len(bottom_nodes)))
357*0.003492891468893659


In [ ]:
#predict_amazon.df_summary_ratio_v.query("movie=='B000N2HD7I'").values[0].tolist()[1]
pd.options.display.max_rows = 1000
predict_amazon.df_summary_ratio_v

In [ ]:
# this takes too much time to run, as well
# The eccentricity of a node v is the maximum distance from v to all other nodes in G.
# Returns: ecc – A dictionary of eccentricity values keyed by node.
# eccentricity(G, v=None, sp=None)
#e_model = nx.eccentricity(max_connected_gr_amazon_movies)
#e_test = nx.eccentricity(max_connected_gr_amazon_movies_VAL)

#print('Eccentricity of MODEL: {}\tTEST: {}'.format(e_model, e_test))

In [ ]:
# below table shows the prediction for threshold>10% so don't pay attention to it
# also tot_rev_m_NEW & OLD will be double checked, seems like a bug
pd.options.display.max_rows = 1000
df_FINAL

In [ ]:
#pd.options.display.max_rows = 35500
df_FINAL_all.query("user=='A22GKUN35R09KN' and movie=='B000J6I0TS'")

In [ ]:
'''
total_reviews = 0
        # total number of reviews that those 100 test users did in dataset-2
        for u in df_FINAL['user'].unique():
            n_movie_reviewed_new = df_FINAL.query("user == '" + u + "'")['n_new_edge'][0]
            total_reviews += n_movie_reviewed_new



for u in df_FINAL['user'].unique():
    print(u)
    print(df_FINAL.query("user == '" + u + "'")['n_new_edge'].values[0])
'''

In [ ]:
len(df_FINAL['movie'].unique())

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
'''
for t in thresholds:
    acc, tpr, fpr, tnr, fnr, ppv, f1 = predict_amazon.final_summary_threshold_based(df_FINAL, t)
    lst_t = [t,  acc, tpr, fpr, tnr, fnr, ppv, f1]
    lst_presicion_based.append(lst_t)
'''

In [ ]:
'''
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] > threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0
        
acc, tpr, fpr, tnr, ppv, f1 = predict_amazon.final_summary(df_FINAL_copy)

print(tpr)

#p_recall = tpr / (tpr + fnr)
#print(recall)
'''

In [ ]:
lst_true_val = df_FINAL.loc[:, 'reality'].values.tolist()
lst_ratio_val = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

fpr, tpr, thresholds = metrics.roc_curve(lst_true_val, lst_ratio_val)  #, pos_label=2
lst_tpr = list(tpr); lst_fpr = list(fpr); lst_thresholds = list(thresholds) 

print(tpr)
print(thresholds)

In [ ]:
'''

y_true = df_FINAL.loc[:, 'reality'].values.tolist()
y_pred = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

roc_auc = roc_auc_score(y_true, y_pred)
print(roc_auc)

#precision = precision_score(y_true, y_pred, average='macro') # 'micro'  'weighted' None  average=None, zero_division=1
#print(precision)

precision_recall = precision_recall_curve(y_true, y_pred)
#print(list(precision_recall))

avg_precision = average_precision_score(y_true, y_pred)
#print(avg_precision)

#lst_tpr = 

#for i in range(len(lst_tpr)):
 #   lst_precision[i] = lst_tpr[i]/(lst_tpr[i] + lst_fpr[i])
    
#print(lst_precision)
'''

In [ ]:
'''
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] >= threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0

tp = len(df_FINAL_copy.query("prediction==1 and reality==1").values)
fp = len(df_FINAL_copy.query("prediction==1 and reality==0").values)
tn = len(df_FINAL_copy.query("prediction==0 and reality==0").values)
fn = len(df_FINAL_copy.query("prediction==0 and reality==1").values)

p_precise = tp / (tp + fp)
p_recall = tp / (tp + fn)

print(p_precise)
print(p_recall)
'''

In [ ]:
#u1=df_FINAL_copy.query("(prediction>0 or reality>0) and n_new_edge>0")
#u1=df_FINAL_copy.query("user=='AR15V2ULA2EYM' and (prediction>0 or reality>0) and n_new_edge>0 ").sort_values(by='ngbr_ratio', ascending=False)
#u1

In [ ]:
## NOTES
'''
Model1: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 80.23
Model2: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 86.89 .. 71.15 .. 
Model3: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 
Model4: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 

'''